In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Wyświetl kod</a>''')

In [2]:
import ipywidgets as widgets
from ipywidgets import Layout
import os
import pandas as pd
from IPython.display import clear_output
style = {'description_width': 'initial'}

In [3]:
kolejka = widgets.BoundedIntText(value=1, min=1, max=30, step=1, description='Wybierz kolejkę:', disabled=False, style=style)
display(kolejka)
button = widgets.Button(description='Zatwierdź',disabled=False,tooltip='Confirm',icon='check') 
output = widgets.Output()
display(button,output)

BoundedIntText(value=1, description='Wybierz kolejkę:', max=30, min=1, style=DescriptionStyle(description_widt…

Button(description='Zatwierdź', icon='check', style=ButtonStyle(), tooltip='Confirm')

Output()

In [4]:
def on_confirm_button_clicked(b):
    with output:
        display_analysis(kolejka.value)
    
button.on_click(on_confirm_button_clicked)

In [5]:
base = r'KolejkiEkstraklasa'

In [6]:
def display_analysis(kolejka):
    clear_output()
    matchdays = []
    for filename in os.listdir(base):
        if int(filename)<kolejka:
            matchday = pd.read_csv(f"{base}/{filename}", header=None)
            matchdays.append(matchday)
    league = pd.concat(matchdays, ignore_index=True)
    g = open("Statystyki/poissonstats-ekstraklasa", "r", encoding='utf-8')
    clubs = [x.strip().split(",")[0] for x in g]
    columns = ["avg_prob", "avg_pos", "diff_points", "diff_goals", "diff_conceded", "clear_pred_amount", 
               "clear_pred_prob", "points"]
    clubs_pred = {}

    for c in clubs:
        clear_pred = 0
        clear_real = 0
        clubs_pred[c] = [0,0,0,0,0,0,0,0]
        points = 0
        for index, row in league.iterrows():
            home = row[0].split('-')[0]
            away = row[0].split('-')[1]
            if c==home or c==away:
                clubs_pred[c][0] += float(row[2])/14
                clubs_pred[c][1] += int(row[3])/14
                if c==row[0].split('-')[0]:
                    a = (int(row[1].split(':')[0])-int(row[4].split(':')[0]))
                    b = (int(row[1].split(':')[1])-int(row[4].split(':')[1]))
                    clubs_pred[c][3] += a
                    clubs_pred[c][4] += b
                    if int(row[4].split(':')[1]) is 0:
                        clear_pred += 1
                        if int(row[1].split(':')[1]) is 0:
                            clear_real +=1
                    if int(row[4].split(':')[0])>int(row[4].split(':')[1]):
                        points+=3
                    elif int(row[4].split(':')[0]) is int(row[4].split(':')[1]):
                        points+=1
                elif c==row[0].strip().split('-')[1]:
                    a = (int(row[1].split(':')[1])-int(row[4].split(':')[1]))
                    b = (int(row[1].split(':')[0])-int(row[4].split(':')[0]))
                    clubs_pred[c][3] += a
                    clubs_pred[c][4] += b
                    if int(row[4].split(':')[0]) is 0:
                        clear_pred += 1
                        if int(row[1].split(':')[0]) is 0:
                            clear_real +=1
                    if int(row[4].split(':')[1])>int(row[4].split(':')[0]):
                        points+=3
                    elif int(row[4].split(':')[1]) is int(row[4].split(':')[0]):
                        points+=1   
        clubs_pred[c][5] = clear_pred
        clubs_pred[c][6] = clear_real/max(clear_pred,1)
        clubs_pred[c][7] = points
        
    league_pred_df = pd.DataFrame.from_dict(clubs_pred, orient='index', columns = columns)
    display(league_pred_df.sort_values("points",ascending=False))